# <center>4.1 Machine Learning con Python: K-NN</center>
## <center> IBM Innovation Lab, Uruguay. </center>

### 1. Introducción
En este notebook se trabajará con un dataset de clientes. Con el objetivo de segmentar la clientela, utilizaremos una técnica de machine learning muy simple pero muy poderosa: _K Nearest Neighbors_.

__¿Qué es K-Nearest Neighbors?__

Es un algoritmo de aprendizaje supervisado que clasifica en base a alguna noción de __cercanía.__ Un punto (en nuestro caso un cliente) va a ser clasificado igual que sus vecinos más cercanos, de dónde el nombre: __nearest neighbors__. 

En otras palabras, la clase de un punto será la indicada por los k puntos más cercanos. En su versión más simple, kNN realiza una simple votación entre los k vecinos más cercanos y al punto objetivo se le asigna la clase mayoritaria. Versiones más sofisticadas de este algoritmo ponderan esa votación teniendo en cuenta las distancias a los k vecinos con funciones denominadas _kernels_.  

Notar que este algoritmo __NO__ tiene fase de aprendizaje. Se pasa directamente a la fase de predicción. Para predecir, se toma el punto desconocido, se computan los k vecinos más cercanos y se determina la clase del punto desconocido. Queda claro que, para bases de datos grandes, este algoritmo puede ser costoso computacionalmente  ya que se deben calcular tantas distancias cómo puntos haya en la base. (Hay formas de optimizar este computo, utilizando métodos de aprendizaje no supervisado cómo __Clustering__).

__Visualización del algoritmo__

<img src = "https://ibm.box.com/shared/static/mgkn92xck0z05v7yjq8pqziukxvc2461.png">

En el caso de la figura el punto a predecir es la estrella roja. 

__¿En que influye el parámetro k?__
- Caso __k = 3__: Clase B.
- Caso __k = 6__: Clase A.
- Caso __k = N__: Empate, se decide aleatoriamente.

Notar que elegir cuántos vecinos se analizan será vital para el correcto funcionamiento del modelo. Más adelante, analizaremos cómo determinar el valor óptimo para k.

Para no re inventar la rueda, importamos las librerías pre hechas. 

In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

ModuleNotFoundError: No module named 'numpy'

### 2. La base de datos.

Una empresa de telecomunicaciones separó a sus clientes en 4 grupos en base a información de los clientes y distintos patrones de uso de los servicios que provee la empresa. 

Queremos saber, dada la información de un posible futuro cliente de la empresa (edad, región, ..),  a que grupo pertence. De esta forma, la empresa podrá customizar sus ofertas/promociones/decisiones. Es un típico problema de __clasificación__. 

Los 4 grupos de usuarios son los siguientes: 
- Servicio Básico
- E-Service
- Servicio Plus
- Servicio Total

In [ ]:
df = pd.read_csv('teleCust1000t.csv')
df.head()

In [ ]:
df.describe()

### 3. Análisis y visualización 

¿Cuantos clientes pertenecen a cada categoría?

In [ ]:
df['custcat'].value_counts()

Utilicemos las técnicas de visualización de los notebooks anteriores.

In [ ]:
df.hist(column='age', bins=20)

In [ ]:
df.hist(column='income', bins=20)

In [ ]:
df.hist(column='tenure', bins=20)

### 4. Pre-procesamiento

### 4.1 Features

Definamos los vectores de entrenamiento X:

In [ ]:
df.columns

Utilizaremos sci-kit learn para crear el modelo. Definamos nuestro conjunto de entrenamiento.

In [ ]:
X = df[['region', 'tenure','age', 'marital', 'address', 'income', 'ed', 'employ','retire', 'gender', 'reside']] .values  #.astype(float)
X[0:5]


What are our lables?

In [ ]:
y = df['custcat'].values
y[0:5]

#### 4.2 Normalización 

En algoritmos cómo K-NN dónde se clasifica en base a distancias, es __imprescindible__ normalizar los datos.

In [ ]:
X = preprocessing.StandardScaler().fit(X).transform(X.astype(float))
X[0:5]

#### 4.3 Train Test Split  

Vamos a separar la base de datos en dos conjuntos, un __conjunto de entrenamiento__ y __un conjunto de test__. Esta separación nos permitirá probar el modelo con datos _similares_ a los de entrenamiento, lo cual es un arma de doble filo ya que podemos terminar sobre ajustandonos a los datos de test.

Para realizar la separación, utilizaremos la función __train_test_split__ de sklearn.model_selection.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

### 5. Entrenamiento y clasificación. 

## K nearest neighbor (K-NN)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

#### 5.1 Entrenamiento

Empezaremos con k = 4.

In [ ]:
k = 4
  
neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
neigh

#### 5.2 Clasificación
Hagamos predicciones sobre el conjunto de test.

In [ ]:
yhat = neigh.predict(X_test)
yhat[0:5]

### 6. ¿Cuán preciso es el modelo?

En un problema de clasificación multiclase, la función __accuracy_score__ de la librería _metrics_ permite estimar cuan cerca está el __ground truth__ de las etiquetas predichas. 

In [ ]:
from sklearn import metrics
print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, yhat))

#### Optimizando hiper parámetros
Repita el proceso para k=6.

Haga doble click __aquí__ para ver la solución.

<!-- :
    
    
k = 6
neigh6 = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
yhat6 = neigh6.predict(X_test)
print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh6.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, yhat6))

-->

#### Cómo elegir el k?
El k en KNN es el numero de vecions a examinar. ¿Cómo elegir el k correcto? Con __Validación__.

Es decir, k será uno de los parámetros que aprenderemos. Para realizar este aprendizaje, dejaramos una parte de los datos de lado. Este conjunto será utilizado para medir la precisión de nuestro modelo. Luego variaremos el k y repetiremos el proceso. Al final, elegiremos el __k__ que tenga mejor _error de validación_. Una variante de este método es el de validación __cruzada__, que es empíricamente mejor que la validación simple.


In [ ]:
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)

    
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])

mean_acc

#### Precisión en función de k.

In [ ]:
plt.plot(range(1,Ks),mean_acc,'g')
plt.fill_between(range(1,Ks),mean_acc - 1 * std_acc,mean_acc + 1 * std_acc, alpha=0.10)
plt.legend(('Accuracy ', '+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Nabors (K)')
plt.tight_layout()
plt.show()

In [ ]:
print( "La mejor precisión es", mean_acc.max(), "con k=", mean_acc.argmax()+1) 